Parsing html of suumo (odawara rental)

In [ ]:
from bs4 import BeautifulSoup as bs
import time
import requests

#User inputs
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=14206&ta=14&cb=7.5&ct=8.5&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="

rounding = -3 # By default, it will round to the nearest 10000 (-3 for 1000, -2 for 100, etc.)

# variables
url += "&page="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

prices = []
for i in range(1, 5):
    try:
        response = requests.get(f"{url}{i}", headers=headers)
        response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
        html_content = response.text
        print(i, html_content)
        soup = bs(html_content, 'html.parser')

        # Find all parent elements with class 'cassetteitem_price cassetteitem_price--rent'
        parent_elements = soup.find_all('span', class_='cassetteitem_price cassetteitem_price--rent')
        # Extract child elements with the specific class within the parent
        
        for parent in parent_elements:
            price = parent.find('span', class_='cassetteitem_other-emphasis ui-text--bold')
            if price:
                price = float(price.text.replace("万円", "")) * 10000
                prices.append(price)
        
        time.sleep(5)
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch page {i}: {e}")
        
print(len(prices))







Data summary

In [ ]:
print("Listing count: ", len(prices))
print("Average price: ", int(sum(prices) / len(prices)), "円")


Data processing

In [ ]:
frequency = {}

for price in prices:
    price = str(round(price, rounding))
    frequency[price] = frequency.get(price, 0) + 1

Data Visualization

In [ ]:
import matplotlib.pyplot as plt

frequency = dict(sorted(frequency.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(20, 10))
plt.bar(frequency.keys(), frequency.values())